# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096853


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:00,  2.17s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:06,  2.47s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.76s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:33,  1.34s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.68it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  4.00it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  4.14it/s]

Rendering models: 100%|██████████| 30/30 [00:13<00:00,  4.50it/s]

not-logged-in-025360e89444727733a2    0.000521
not-logged-in-3b0b929f4475fd276561    0.016933
CoreyGallegos23                       0.001067
orangeninja27                         0.000464
Jonas_Cross                           0.000476
jnarayanbvg                           0.002011
TateK                                 0.000678
mwalsh868                             0.000646
izzyathogwarts                        0.138190
not-logged-in-c6917d9b19f41e6c2995    0.003104
ferlin1998                            0.000713
ElisabethB                            0.000408
girlsclub2018                         0.004980
not-logged-in-314d48142a3a122ba666    0.002011
tayyib456                             0.007528
pangeli5                              0.002383
not-logged-in-812b279e0836b083d452    0.001654
Saharisunshine                        0.002100
Lavadude                              0.021908
not-logged-in-5a859927574f86e26115    0.010056
equidad1                              0.128275
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())